In [50]:
import pandas as pd
import numpy as np

import scipy
import statsmodels.formula.api as smf
import statistics as st 
import pandas as pd
import matplotlib.pyplot as plt

import requests
from geopy.geocoders import Nominatim
from astral import LocationInfo
import datetime
from astral.sun import sun
from astral import LocationInfo

from time import time

In [51]:
cd

/Users/karatechop


In [52]:
cd /Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022\ \(CSV\)

/Users/karatechop/Documents/GitHub/senior-thesis-pretextual-stops/Data/RIPA-DATA-CA-as-of-2022 (CSV)


In [53]:
working_main = pd.read_csv('working_main.csv')
print(working_main.columns,'\n',working_main.shape,'\n',len(working_main.time))

Index(['date', 'time', 'sex', 'race', 'type', 'duration', 'city'], dtype='object') 
 (2463454, 7) 
 2463454


In [54]:
working_main.type.value_counts()

Moving        1831696
Equipment      369873
Non-Moving     261885
Name: type, dtype: int64

In [55]:
time = []
for i in range(len(working_main['time'])): 
    current = working_main.loc[i,'time']
    
    if len(working_main.loc[i,'time']) < 8:
        time.append(current + ':00')
    elif len(working_main.loc[i,'time']) > 8:
        time.append(current[11:19])
    else:
        time.append(current)
        
working_main['time'] = time
working_main['time'] = pd.to_datetime(working_main['time'],format="%H:%M:%S")
print('len of time column is:',len(working_main.time),'\n sum of null time values is:',working_main['time'].isna().sum())
working_main

len of time column is: 2463454 
 sum of null time values is: 0


,date,time,sex,race,type,duration,city
0,12/09/2021,1900-01-01 13:00:00,Male,hispanic/latino(a),Non-Moving,5.0,ALL-SJPD-AB953
1,04/26/21,1900-01-01 23:00:00,Male,black/african american,Moving,60.0,ALL-SJPD-AB953
2,04/15/21,1900-01-01 11:35:00,Female,black/african american,Equipment,10.0,ALL-SJPD-AB953
3,04/01/21,1900-01-01 01:53:00,Male,asian,Non-Moving,100.0,ALL-SJPD-AB953
4,12/20/2021,1900-01-01 16:07:00,Female,white,Moving,10.0,ALL-SJPD-AB953
...,...,...,...,...,...,...,...
2463449,11/02/2021,1900-01-01 00:19:00,Female,hispanic/latino(a),Non-Moving,74.0,ALL-LAPD-AB953
2463450,11/02/2021,1900-01-01 00:14:00,Male,hispanic/latino(a),Non-Moving,81.0,ALL-LAPD-AB953
2463451,11/01/2021,1900-01-01 01:00:00,Male,hispanic/latino(a),Moving,13.0,ALL-LAPD-AB953
2463452,11/01/2021,1900-01-01 01:00:00,Male,hispanic/latino(a),Non-Moving,13.0,ALL-LAPD-AB953


In [57]:
working_main['city'].isna().sum()
working_main['city'].value_counts()

ALL-LAPD-AB953          1602977
ALL-RIVERSIDE-AB953      314497
SDPD                     246655
ALL-SFPD-AB953           130977
ALL-OAKLAND_PD-AB953      68527
ALL-SJPD-AB953            50443
ALL-LBPD-AB953            36670
Berkeley PD                6466
ALL-MARIN-AB953            4746
Davis PD                   1496
Name: city, dtype: int64

In [58]:
working_main.rename(columns={'city':'agency'},inplace=True)

def city_from_agency_check(df):
    city_str = []
    lat_str = []
    lon_str = []
    wave_str = []
    
    for i in df['agency']:
        if 'ALL-SJPD-AB953' in i:
            city_str.append('San Jose')
            lat_str.append(37.3387)
            lon_str.append(-121.8853)
            wave_str.append(2)
        elif 'ALL-LBPD-AB953' in i:
            city_str.append('Long Beach')
            lat_str.append(33.7701)
            lon_str.append(-118.1937)
            wave_str.append(2)
        elif 'ALL-OAKLAND_PD-AB953' in i:
            city_str.append('Oakland')
            lat_str.append(37.8044)
            lon_str.append(-122.2712)
            wave_str.append(2)
        elif 'Berkeley PD' in i:
            city_str.append('Berkeley')
            lat_str.append(37.8715)
            lon_str.append(-122.2730)
            wave_str.append(3)
        elif 'ALL-MARIN-AB953' in i:
            city_str.append('Marin')
            lat_str.append(38.0834)
            lon_str.append(-122.7633)
            wave_str.append(3)
        elif 'Davis PD' in i:
            city_str.append('Davis')
            lat_str.append(38.5449)
            lon_str.append(-121.7405)
            wave_str.append(3)
        elif 'ALL-RIVERSIDE-AB953' in i:
            city_str.append('Riverside')
            lat_str.append(33.9806)
            lon_str.append(-117.3755)
            wave_str.append(1)
        elif 'SDPD' in i:
            city_str.append('San Diego')
            lat_str.append(32.7157)
            lon_str.append(-117.1611)
            wave_str.append(1)
        elif 'ALL-SFPD-AB953' in i:
            city_str.append('San Francisco')
            lat_str.append(37.7749)
            lon_str.append(-122.4194)
            wave_str.append(1)
        elif 'ALL-LAPD-AB953' in i:
            city_str.append('Los Angeles')
            lat_str.append(34.0522)
            lon_str.append(-118.2437)
            wave_str.append(1)
        
    df['city']=city_str
    df['lat']=lat_str
    df['lon']=lon_str
    df['wave']=wave_str
    
    pd.option_context('display.precision', 10)
    
    return df

working_main = city_from_agency_check(working_main)
working_main

,date,time,sex,race,type,duration,agency,city,lat,lon,wave
0,12/09/2021,1900-01-01 13:00:00,Male,hispanic/latino(a),Non-Moving,5.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2
1,04/26/21,1900-01-01 23:00:00,Male,black/african american,Moving,60.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2
2,04/15/21,1900-01-01 11:35:00,Female,black/african american,Equipment,10.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2
3,04/01/21,1900-01-01 01:53:00,Male,asian,Non-Moving,100.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2
4,12/20/2021,1900-01-01 16:07:00,Female,white,Moving,10.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2
...,...,...,...,...,...,...,...,...,...,...,...
2463449,11/02/2021,1900-01-01 00:19:00,Female,hispanic/latino(a),Non-Moving,74.0,ALL-LAPD-AB953,Los Angeles,34.0522,-118.2437,1
2463450,11/02/2021,1900-01-01 00:14:00,Male,hispanic/latino(a),Non-Moving,81.0,ALL-LAPD-AB953,Los Angeles,34.0522,-118.2437,1
2463451,11/01/2021,1900-01-01 01:00:00,Male,hispanic/latino(a),Moving,13.0,ALL-LAPD-AB953,Los Angeles,34.0522,-118.2437,1
2463452,11/01/2021,1900-01-01 01:00:00,Male,hispanic/latino(a),Non-Moving,13.0,ALL-LAPD-AB953,Los Angeles,34.0522,-118.2437,1


In [59]:
working_main.agency.value_counts()

ALL-LAPD-AB953          1602977
ALL-RIVERSIDE-AB953      314497
SDPD                     246655
ALL-SFPD-AB953           130977
ALL-OAKLAND_PD-AB953      68527
ALL-SJPD-AB953            50443
ALL-LBPD-AB953            36670
Berkeley PD                6466
ALL-MARIN-AB953            4746
Davis PD                   1496
Name: agency, dtype: int64

In [60]:
# working_main.to_csv('working_main_expanded.csv')

In [61]:
working_main_expanded = working_main

In [62]:
# working_main_expanded = pd.read_csv('working_main_expanded.csv')

In [63]:
working_main_expanded.date = pd.to_datetime(working_main_expanded.date)

In [64]:
working_main_expanded.dtypes

date                object
time        datetime64[ns]
sex                 object
race                object
type                object
duration           float64
agency              object
city                object
lat                float64
lon                float64
wave                 int64
dtype: object

In [65]:
# %%timeit
# start = time()

sunset_list = []
counter = 0



for i in working_main_expanded['date']:
    year = i.year
    month = i.month
    day = i.day

    name = working_main_expanded['city'][counter]
    lat = working_main_expanded['lat'][counter]
    lon = working_main_expanded['lon'][counter]

    loc = LocationInfo(name=name, region='CA, USA', timezone='America/Los_Angeles', #https://en.wikipedia.org/wiki/List_of_tz_database_time_zones
                       latitude=lat, longitude=lon)
    


    s = sun(loc.observer, date=datetime.date(year, month, day), tzinfo=loc.timezone)
    for key in ['sunset']:
        sunset_list.append(s[key].time())
        
    counter+=1
    
sunset_list2 = [str(i)[:8] for i in sunset_list]  
working_main_expanded['sunsettime'] = sunset_list2
# print(f'that whole process took {(time()-start)/60} minutes')
# sunset_list

In [66]:
date_to_subtract = datetime.datetime(1900, 1, 1)

working_main_expanded['mins_sunset'] = [round(((datetime.datetime.strptime(i, '%H:%M:%S') - date_to_subtract).total_seconds() / 60.0),0) 
                                           for i in sunset_list2]
working_main_expanded['mins_stop_time'] = [round(((i - date_to_subtract).total_seconds()/60),0) 
                                           for i in working_main_expanded['time']]
working_main_expanded['mins_after_sunset'] = working_main_expanded['mins_stop_time'] - working_main_expanded['mins_sunset']
working_main_expanded['day_in_year'] = [i.day for i in working_main_expanded['date']]
working_main_expanded.head(5)

,date,time,sex,race,type,duration,agency,city,lat,lon,wave,sunsettime,mins_sunset,mins_stop_time,mins_after_sunset,day_in_year
0,2021-12-09 00:00:00,1900-01-01 13:00:00,Male,hispanic/latino(a),Non-Moving,5.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,16:49:31,1010.0,780.0,-230.0,9
1,2021-04-26 00:00:00,1900-01-01 23:00:00,Male,black/african american,Moving,60.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,19:52:49,1193.0,1380.0,187.0,26
2,2021-04-15 00:00:00,1900-01-01 11:35:00,Female,black/african american,Equipment,10.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,19:42:53,1183.0,695.0,-488.0,15
3,2021-04-01 00:00:00,1900-01-01 01:53:00,Male,asian,Non-Moving,100.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,19:30:20,1170.0,113.0,-1057.0,1
4,2021-12-20 00:00:00,1900-01-01 16:07:00,Female,white,Moving,10.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,16:52:49,1013.0,967.0,-46.0,20


In [67]:
# import json
# data = {}
# counter = 0 

# for i in sunset_list:
#     data[counter] = str(i)
#     counter += 1
    
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(data, f, ensure_ascii=False, indent=4)

In [68]:
# cols = ['date', 'time', 'sex', 'race', 'type', 'duration', 'agency', 'city',
#        'lat', 'lon', 'wave', 'sunsettime','mins_stop_time','mins_sunset','mins_until_sunset']
# working_main_expanded = working_main_expanded[cols]
working_main_expanded

,date,time,sex,race,type,duration,agency,city,lat,lon,wave,sunsettime,mins_sunset,mins_stop_time,mins_after_sunset,day_in_year
0,2021-12-09 00:00:00,1900-01-01 13:00:00,Male,hispanic/latino(a),Non-Moving,5.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,16:49:31,1010.0,780.0,-230.0,9
1,2021-04-26 00:00:00,1900-01-01 23:00:00,Male,black/african american,Moving,60.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,19:52:49,1193.0,1380.0,187.0,26
2,2021-04-15 00:00:00,1900-01-01 11:35:00,Female,black/african american,Equipment,10.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,19:42:53,1183.0,695.0,-488.0,15
3,2021-04-01 00:00:00,1900-01-01 01:53:00,Male,asian,Non-Moving,100.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,19:30:20,1170.0,113.0,-1057.0,1
4,2021-12-20 00:00:00,1900-01-01 16:07:00,Female,white,Moving,10.0,ALL-SJPD-AB953,San Jose,37.3387,-121.8853,2,16:52:49,1013.0,967.0,-46.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463449,2021-11-02 00:00:00,1900-01-01 00:19:00,Female,hispanic/latino(a),Non-Moving,74.0,ALL-LAPD-AB953,Los Angeles,34.0522,-118.2437,1,17:58:32,1079.0,19.0,-1060.0,2
2463450,2021-11-02 00:00:00,1900-01-01 00:14:00,Male,hispanic/latino(a),Non-Moving,81.0,ALL-LAPD-AB953,Los Angeles,34.0522,-118.2437,1,17:58:32,1079.0,14.0,-1065.0,2
2463451,2021-11-01 00:00:00,1900-01-01 01:00:00,Male,hispanic/latino(a),Moving,13.0,ALL-LAPD-AB953,Los Angeles,34.0522,-118.2437,1,17:59:28,1079.0,60.0,-1019.0,1
2463452,2021-11-01 00:00:00,1900-01-01 01:00:00,Male,hispanic/latino(a),Non-Moving,13.0,ALL-LAPD-AB953,Los Angeles,34.0522,-118.2437,1,17:59:28,1079.0,60.0,-1019.0,1


In [69]:
working_main_expanded.to_csv('working_main_expanded.csv',index=False)

In [70]:
working_main_expanded.type.value_counts()

Moving        1831696
Equipment      369873
Non-Moving     261885
Name: type, dtype: int64